In [1]:
!pip install evaluate sacrebleu transformers==4.28.0 #Avoid PartialState error with accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("HF_KEY")
login(secret_value)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
#Load in EN-FR subset of OPUS books
from datasets import load_dataset, load_dataset_builder,concatenate_datasets
books = load_dataset("ethansimrm/OpusTrain") #Using the same dataset as Colab for consistency
booksTest = load_dataset("ethansimrm/OpusTest") #Using the same dataset as Colab for consistency

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/ethansimrm--OpusTrain-0e62f5268a5f0ef3/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/ethansimrm--OpusTest-e350203b885f5992/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
#ds_builder = load_dataset_builder("rotten_tomatoes")
#ds_builder = load_dataset_builder("wmt14", 'fr-en')
#ds_builder.info.description
#ds_builder.info.features

In [5]:
#Entirety of OPUS books is a training dataset, so split into train:test with 80:20.
#books = books["train"].train_test_split(test_size=0.2)

In [6]:
#Inspect data; the split is random.
books['train'][0]

{'id': '67130',
 'translation': {'en': 'Since he had grown to manhood they no longer said in so many words:"Look at Jean and follow his example," but every time he heard them say"Jean did this--Jean does that," he understood their meaning and thehint the words conveyed.',
  'fr': "Depuis qu'il était homme, on ne lui disait plus: «Regarde Jean etimite-le!» mais chaque fois qu'il entendait répéter: «Jean a fait ceci,Jean a fait cela,» il comprenait bien le sens et l'allusion cachés sousces paroles."}}

In [7]:
#Tokenise data - we must load the correct tokeniser for our model before we input parameters. This is based on SentencePiece.
from transformers import AutoTokenizer
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
#NB: As long as data looks like {"input" : "XXXX" , "target" : "YYYY"}, it can be processed.

In [8]:
#Specify a preprocessing function which allows us to tokenise source and target languages correctly AND prime T5 with the correct prompt before each sentence
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]] 
    #We are essentially querying the dictionary here, whereby books["train"]["translation"][0]["en"] references the first English sentence above.
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True) #We will truncate sentences longer than 128 words long
    return model_inputs

#books["train"]["translation"][0]["en"] will give us the first English sentence

In [9]:
#Apply this function over the training dataset over multiple elements simultaneously using the batched = True argument.
tokenized_books = books.map(preprocess_function, batched=True)
tokenized_booksTest = booksTest.map(preprocess_function, batched=True)

  0%|          | 0/102 [00:00<?, ?ba/s]

  0%|          | 0/26 [00:00<?, ?ba/s]

In [10]:
#Create a batch of examples and dynamically pad to hit length of longest sentence per batch
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)
#Ignore the tensorflow linkage warning; Kaggle's conda forge does not have it

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [11]:
#Load evaluation method
import evaluate
metric = evaluate.load("sacrebleu")

In [12]:
import numpy as np

def postprocess_text(preds, labels): 
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) #Convert back into words

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) #Ignore padded labels added by the data collator to the test set
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True) 

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels) #Remove leading and trailing spaces

    result = metric.compute(predictions=decoded_preds, references=decoded_labels) #BLEU score for provided input and references
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens) #Compute mean prediction length
    result = {k: round(v, 4) for k, v in result.items()} #Round score to 4dp
    return result

In [13]:
#Ready to download model
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint) #Model is 242MB in size

In [15]:
import torch

training_args = Seq2SeqTrainingArguments( #Collects hyperparameters
    output_dir="test_t5_small_example_kaggle3",
    evaluation_strategy="epoch", #Evaluates at the end of each epoch
    learning_rate=2e-5, #Initial learning rate for AdamW
    per_device_train_batch_size=16, #Minibatch learning
    per_device_eval_batch_size=16, #Batch size for evaluation
    weight_decay=0.01, #Weight decay for loss computation; Loss = Loss + WD * sum (weights squared)
    save_total_limit=3, #Number of checkpoints to save
    num_train_epochs=2,
    predict_with_generate=True, #Use with ROUGE/BLEU and other translation metrics (see below)
    fp16=True, #Remove fp16 = True if not using CUDA
    push_to_hub=True,
)

trainer = Seq2SeqTrainer( #Saves us from writing our own training loops
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_booksTest["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#However, these metrics require that we generate some text with the model rather than a single forward pass as with e.g. classification. 
#The Seq2SeqTrainer allows for the use of the generate method when setting predict_with_generate=True which will generate text for each sample in the evaluation set. 
#That means we evaluate generated text within the compute_metric function. We just need to decode the predictions and labels first.

Cloning https://huggingface.co/ethansimrm/test_t5_small_example_kaggle3 into local empty directory.


In [16]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.847500,1.636042,5.495200,17.611800
2,1.814900,1.612160,5.681600,17.595500


TrainOutput(global_step=12710, training_loss=1.8734240756271205, metrics={'train_runtime': 3241.1958, 'train_samples_per_second': 62.735, 'train_steps_per_second': 3.921, 'total_flos': 5017466336575488.0, 'train_loss': 1.8734240756271205, 'epoch': 2.0})

In [17]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/231M [00:00<?, ?B/s]

Upload file runs/May25_22-25-02_2a7868de4d98/events.out.tfevents.1685053531.2a7868de4d98.28.0:   0%|          …

To https://huggingface.co/ethansimrm/test_t5_small_example_kaggle3
   f3c5813..b01232f  main -> main

To https://huggingface.co/ethansimrm/test_t5_small_example_kaggle3
   b01232f..2cf1772  main -> main



'https://huggingface.co/ethansimrm/test_t5_small_example_kaggle3/commit/b01232fff60aa1c3451a1880dcb7f110afef79b4'

In [18]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

In [19]:
from transformers import pipeline

translator = pipeline("translation", model="test_t5_small_example_kaggle3")
translator(text)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/__init__.py:958: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


[{'translation_text': 'Legumes teilen Ressourcen mit Stickstoff-fixierenden Bakterien.'}]